# Implementing SCAN*PRO Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data_path = 'data.csv'
data = pd.read_csv(data_path)

# Data cleaning and preparation
# Removing an unnecessary 'Unnamed: 0' column 
if 'Unnamed: 0' in data.columns:
    data.drop('Unnamed: 0', axis=1, inplace=True)
    



In [2]:
# Generating summary statistics
summary_stats = data[['PRICE', 'BASE_PRICE','FEATURE', 'DISPLAY', 'TPR_ONLY']].describe()
print(summary_stats)


               PRICE     BASE_PRICE        FEATURE        DISPLAY  \
count  538435.000000  538435.000000  538435.000000  538435.000000   
mean        3.385317       3.605310       0.084361       0.110145   
std         1.560662       1.632824       0.277929       0.313071   
min         0.000000       0.550000       0.000000       0.000000   
25%         2.370000       2.500000       0.000000       0.000000   
50%         2.990000       3.180000       0.000000       0.000000   
75%         4.490000       4.590000       0.000000       0.000000   
max        11.460000      11.460000       1.000000       1.000000   

            TPR_ONLY  
count  538435.000000  
mean        0.133894  
std         0.340538  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         0.000000  
max         1.000000  


In [3]:
# Calculating the correlation matrix
correlation_matrix = data[['PRICE', 'BASE_PRICE', 'FEATURE', 'DISPLAY', 'TPR_ONLY']].corr()
print(correlation_matrix)


               PRICE  BASE_PRICE   FEATURE   DISPLAY  TPR_ONLY
PRICE       1.000000    0.953582 -0.010490 -0.069339 -0.181595
BASE_PRICE  0.953582    1.000000  0.155153  0.045255 -0.050310
FEATURE    -0.010490    0.155153  1.000000  0.407148 -0.119345
DISPLAY    -0.069339    0.045255  0.407148  1.000000 -0.138330
TPR_ONLY   -0.181595   -0.050310 -0.119345 -0.138330  1.000000


In [4]:
#VIF calculation for predictors
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
# Selecting the predictors for VIF calculation
predictors = data[['PRICE', 'BASE_PRICE', 'FEATURE', 'DISPLAY', 'TPR_ONLY']]
X = sm.add_constant(predictors)

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


      feature        VIF
0       const   6.921443
1       PRICE  27.022563
2  BASE_PRICE  26.565330
3     FEATURE   1.841085
4     DISPLAY   1.360911
5    TPR_ONLY   1.725373


In [5]:
'''Fitting the model'''

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error


data = pd.read_csv('data.csv') 

# Convert 'WEEK_END_DATE' column to datetime format for time series analysis
data['WEEK_END_DATE'] = pd.to_datetime(data['WEEK_END_DATE'])

# Sorting the dataset in chronological order is crucial for time series analysis
data.sort_values(by='WEEK_END_DATE', inplace=True)


relevant_columns = ['WEEK_END_DATE', 'UNITS', 'PRICE', 'FEATURE', 'DISPLAY', 'TPR_ONLY', 'CATEGORY']

# Function to convert boolean columns to integer format
def convert_bool_to_int(df):
    # Iterate through each column in the DataFrame
    for col in df.select_dtypes(include=['bool']).columns:
        # Convert boolean column to integer type
        df[col] = df[col].astype(int)
    return df

# Loop through each unique category in the dataset
for category in data['CATEGORY'].unique():
    print(f"\n\n=== Category: {category} ===")
    
    # Filter data for the current category
    category_data = data.loc[data['CATEGORY'] == category, relevant_columns].copy()

    # Apply log transformation to 'UNITS' and 'PRICE' to normalize data distribution
    category_data['log_UNITS'] = np.log(category_data['UNITS'] + 1)
    category_data['log_PRICE'] = np.log(category_data['PRICE'] + 0.01)

    # Create interaction terms to explore combined effects on sales
    category_data['log_PRICE_FEATURE'] = category_data['log_PRICE'] * category_data['FEATURE']
    category_data['log_PRICE_DISPLAY'] = category_data['log_PRICE'] * category_data['DISPLAY']
    category_data['log_PRICE_TPR_ONLY'] = category_data['log_PRICE'] * category_data['TPR_ONLY']

    # Extract month from 'WEEK_END_DATE' to capture seasonal effects
    category_data['month'] = category_data['WEEK_END_DATE'].dt.month
    
    # Generate dummy variables for months to use in the model
    month_dummies = pd.get_dummies(category_data['month'], prefix='month', drop_first=True)
    
    # Merge the original data with the month dummy variables
    category_data = pd.concat([category_data, month_dummies], axis=1)

    # Define the list of predictors to be used in the model
    predictors = ['log_PRICE', 'FEATURE', 'DISPLAY', 'TPR_ONLY', 'log_PRICE_FEATURE', 
                  'log_PRICE_DISPLAY', 'log_PRICE_TPR_ONLY'] + list(month_dummies.columns)

    # Ensure all boolean columns are converted to integer format
    category_data = convert_bool_to_int(category_data)

    # Define the independent variables (X) and the dependent variable (y)
    X = category_data[predictors]
    y = category_data['log_UNITS']

    # Add a constant term to the predictors for the intercept in the model
    X = sm.add_constant(X)

    # Manually split the data based on index: 80% for training, 20% for testing
    split_idx = int(len(category_data) * 0.8)
    
    # Training data
    X_train = X.iloc[:split_idx]
    y_train = y.iloc[:split_idx]
    
    # Testing data
    X_test = X.iloc[split_idx:]
    y_test = y.iloc[split_idx:]

    # Fit the Ordinary Least Squares (OLS) model with the training data
    
    model = sm.OLS(y_train, X_train).fit()
# Predicting on the test set
    y_pred = model.predict(X_test)

    # Calculate RMSE for the test set
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    
    print(model.summary())
    print(f"RMSE for {category}: {rmse}")
    




=== Category: BAG SNACKS ===
                            OLS Regression Results                            
Dep. Variable:              log_UNITS   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     2634.
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        06:14:18   Log-Likelihood:            -1.2829e+05
No. Observations:              104084   AIC:                         2.566e+05
Df Residuals:                  104065   BIC:                         2.568e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const

In [6]:
'''Training on the entire dataset and then calculating the rsme'''
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error


data = pd.read_csv('data.csv')  

# Convert 'WEEK_END_DATE' column to datetime format for time series analysis
data['WEEK_END_DATE'] = pd.to_datetime(data['WEEK_END_DATE'])

# Sorting the dataset in chronological order is crucial for time series analysis
data.sort_values(by='WEEK_END_DATE', inplace=True)

# Specify columns relevant for the analysis to ensure data consistency
relevant_columns = ['WEEK_END_DATE', 'UNITS', 'PRICE', 'FEATURE', 'DISPLAY', 'TPR_ONLY', 'CATEGORY']

# Function to convert boolean columns to integer format
def convert_bool_to_int(df):
    for col in df.select_dtypes(include=['bool']).columns:
        df[col] = df[col].astype(int)
    return df

# Loop through each unique category in the dataset
for category in data['CATEGORY'].unique():
    print(f"\n\n=== Category: {category} ===")
    
    # Filter data for the current category
    category_data = data.loc[data['CATEGORY'] == category, relevant_columns].copy()

    # Apply log transformation to 'UNITS' and 'PRICE' to normalize data distribution
    category_data['log_UNITS'] = np.log(category_data['UNITS'] + 1)
    category_data['log_PRICE'] = np.log(category_data['PRICE'] + 0.01)

    # Create interaction terms to explore combined effects on sales
    category_data['log_PRICE_FEATURE'] = category_data['log_PRICE'] * category_data['FEATURE']
    category_data['log_PRICE_DISPLAY'] = category_data['log_PRICE'] * category_data['DISPLAY']
    category_data['log_PRICE_TPR_ONLY'] = category_data['log_PRICE'] * category_data['TPR_ONLY']

    # Extract month from 'WEEK_END_DATE' to capture seasonal effects
    category_data['month'] = category_data['WEEK_END_DATE'].dt.month
    month_dummies = pd.get_dummies(category_data['month'], prefix='month', drop_first=True)
    
    # Merge the original data with the month dummy variables
    category_data = pd.concat([category_data, month_dummies], axis=1)

    # Ensure all boolean columns are converted to integer format
    category_data = convert_bool_to_int(category_data)

    # Define the list of predictors to be used in the model
    predictors = ['log_PRICE', 'FEATURE', 'DISPLAY', 'TPR_ONLY', 'log_PRICE_FEATURE', 'log_PRICE_DISPLAY', 'log_PRICE_TPR_ONLY'] + list(month_dummies.columns)

    # Define the independent variables (X) and the dependent variable (y)
    X = category_data[predictors]
    y = category_data['log_UNITS']

    # Add a constant term to the predictors for the intercept in the model
    X_with_constant = sm.add_constant(X)

    # Fit the Ordinary Least Squares (OLS) model with the entire data
    model = sm.OLS(y, X_with_constant).fit()

    # Print the summary statistics of the fitted model
    print(model.summary())

    # Predict the log_UNITS for the entire dataset
    y_pred_entire = model.predict(X_with_constant)

    # Calculate RMSE for the entire dataset predictions
    rmse_entire = np.sqrt(mean_squared_error(y, y_pred_entire))
    print(f"RMSE for the entire dataset: {rmse_entire}")




=== Category: BAG SNACKS ===
                            OLS Regression Results                            
Dep. Variable:              log_UNITS   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     3234.
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        06:14:19   Log-Likelihood:            -1.6034e+05
No. Observations:              130106   AIC:                         3.207e+05
Df Residuals:                  130087   BIC:                         3.209e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const